In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Hide TensorFlow Warning due to using GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Dense, Flatten, MaxPooling2D, Conv2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
from matplotlib import pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, KFold


In [2]:
# Dataset configuration
data_dir = "../dataset/wadaba"
img_height, img_width = 150, 150
batch_size = 32
dataset_label_mode = 'int'
dataset_color_mode = 'rgb'

# Model configuration
no_epochs = 10
num_folds = 5
optimizer = tf.keras.optimizers.legacy.Adam()
loss = keras.losses.SparseCategoricalCrossentropy()
verbosity = 1

# Data shape
input_shape = (img_height, img_width, 3)

In [3]:
ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode=dataset_label_mode,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(img_height, img_width),
)

Found 4000 files belonging to 7 classes.


2023-06-14 03:22:46.724977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
# Preprocess
def preprocess(images, labels):
    return tf.keras.applications.mobilenet_v2.preprocess_input(images), labels

# Apply preprocess
ds = ds.map(preprocess)

In [5]:
x = np.concatenate([x for x, y in ds], axis=0)
y = np.concatenate([y for x, y in ds], axis=0)

In [6]:
input_train, input_test, target_train, target_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [7]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

In [8]:
# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

In [14]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    base_model = MobileNetV2(
        weights='imagenet',
        input_shape=input_shape,
        include_top=False
    )
    input_ = tf.keras.Input(shape=(150, 150, 3))

    # Set the feature extractor layer
    x = base_model(input_, training=False)

    # Set the pooling layer
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # Set the final layer with sigmoid activation function
    output_ = tf.keras.layers.Dense(7, activation='softmax')(x)

    # Create the new model object
    model = tf.keras.Model(input_, output_)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(), 
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Increase fold number
    fold_no = fold_no + 1

    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Loss: {np.mean(loss_per_fold)}')
    print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/10
100/100 [==============================] - 55s 500ms/step - loss: 1.5725 - accuracy: 0.5047
Epoch 2/10
100/100 [==============================] - 49s 493ms/step - loss: 1.2944 - accuracy: 0.5356
Epoch 3/10
100/100 [==============================] - 49s 494ms/step - loss: 1.2931 - accuracy: 0.5356
Epoch 4/10
 21/100 [=====>........................] - ETA: 39s - loss: 1.2969 - accuracy: 0.5432

KeyboardInterrupt: 